<a href="https://colab.research.google.com/github/umslengineering/EE1108/blob/main/EE1108_ex8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Project 4: Homemade WhatsApp with Python

A chat server is the central coordinator (the “brain” of the system).

It:

👉 Accepts connections from multiple users
👉 Receives messages from one user
👉 Sends (broadcasts or routes) messages to other users
👉 Keeps track of who is connected

Think of it like:

🧠 Post office / traffic controller

Everyone sends messages to the server, and the server distributes them.

In [ ]:
# chat_server.py

#	socket: This module allows your program to communicate over the network (TCP/IP)
# threading: This allows multiple clients to connect and communicate simultaneously, without freezing the server.
import socket
import threading


# HOST: The IP address the server will listen on.
# "127.0.0.1" means only the local computer can connect.
# If you want other computers on the same WiFi to connect,
# you’ll use the server’s actual IP (0.0.0.0 or 192.168.x.x).

# PORT: The network port the server will use. All clients must use the same port.
HOST = "127.0.0.1"   # localhost
PORT = 5555

# socket.socket(socket.AF_INET, socket.SOCK_STREAM): Creates a TCP/IP socket.
# AF_INET = IPv4
# SOCK_STREAM = TCP (reliable connection)
# server.bind((HOST, PORT)): Tells the server to listen on the IP/port you specified.
# server.listen(): Starts listening for incoming client connections.
server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server.bind((HOST, PORT))
server.listen()

# clients: List to store all connected client sockets.
# usernames: List to store the usernames of all connected clients.
# These lists are parallel: clients[i] corresponds to usernames[i].
clients = []
usernames = []

# broadcast is a helper function that sends a message to everyone connected.
# This is how messages from one user get delivered to all others.
def broadcast(message):
    """Send message to all clients."""
    for client in clients:
        client.send(message)

#	handle_client(client) handles one client in a separate thread.
# client.recv(1024): Receives up to 1024 bytes of data from that client.
# broadcast(message): Sends the received message to all other clients.
# If there is an error (client disconnects), we:
#	1.	Find the client’s index
#	2.	Remove them from clients and usernames lists
#	3.	Close the connection
#	4.	Inform everyone that the user left the chat
#	5.	Break the loop (stop the thread)
def handle_client(client):
    while True:
        try:
            message = client.recv(1024)
            broadcast(message)
        except:
            index = clients.index(client)
            clients.remove(client)
            client.close()
            username = usernames[index]
            broadcast(f"{username} left the chat.".encode("utf-8"))
            usernames.remove(username)
            break

# receive_connections() continuously waits for new clients.
# server.accept(): Pauses until a new client connects, returns a socket for that client and its address.
# client.send("USERNAME"): Asks the client to send its username.
# username = client.recv(1024).decode("utf-8"): Receives the username.
# Adds the client and username to the respective lists.
# Prints the username for server-side logs.
# broadcast(f"{username} joined the chat!"): Lets everyone know a new user joined.
# threading.Thread(target=handle_client, args=(client,)):
#                   Starts a new thread to handle messages from this client.
# This ensures the server can handle many clients at once without freezing.
#
def receive_connections():
    print("Server running...")

    while True:
        client, address = server.accept()
        print(f"Connected with {address}")

        client.send("USERNAME".encode("utf-8"))
        username = client.recv(1024).decode("utf-8")

        usernames.append(username)
        clients.append(client)

        print(f"Username is {username}")
        broadcast(f"{username} joined the chat!".encode("utf-8"))

        thread = threading.Thread(target=handle_client, args=(client,))
        thread.start()

#	Starts the server loop that waits for clients to connect.
# This is the main entry point of the server.
receive_connections()

Notice: For thread = threading.Thread(target=handle_client, args=(client,)) line of code:

* target is the function the thread will run.
	•	Here, it’s handle_client, which expects one argument: the client socket.

* args=(client,)
	•	args is a tuple of arguments to pass to the target function.
	•	Notice the comma: (client,) is a tuple with one item.

A chat client is the program each user runs.

It:

👉 Connects to the server
👉 Sends messages typed by the user
👉 Receives messages from others
👉 Displays chat interface

Think of it like:

📱 WhatsApp app on your phone.

In [ ]:
# chat_client.py

# Allows the program to communicate over network connections (TCP/IP). Needed to connect to the chat server
# Allows running multiple tasks simultaneously. Since we want to send and receive at the same time!
import socket
import threading

#	HOST = server IP address.
# 127.0.0.1 means connect to server running on same machine.
# PORT = communication channel number.
# Must match server settings.
HOST = "127.0.0.1"
PORT = 5555

username = input("Choose your username: ")

# Creates a TCP socket.
# AF_INET = IPv4.
# SOCK_STREAM = TCP (reliable connection).
client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

#	Establishes connection to chat server.
#	If server is not running → connection fails.
client.connect((HOST, PORT))

#	Defines function responsible for listening to server messages.
def receive_messages():
    while True: # infinite loop, client always listens for the incoming messages
        try:
            message = client.recv(1024).decode("utf-8") # Waits for message from server. Receives up to 1024 bytes. Converts binary data → readable text.

            #handle handshake
            if message == "USERNAME": #	Server sends “USERNAME” when a new client connects. This is a signal asking for the user’s name.
                client.send(username.encode("utf-8")) # Sends username back to server. Encodes text into bytes for network transmission.
            else:
                print(message) # If message is normal chat text → display it.

        except: #handling connection errors
            print("Connection closed.")
            client.close() #close socket
            break

# function: sending message
def write_messages():
    while True:
        msg = input("") #	Wait for user to type message.
        message = f"{username}: {msg}" # Adds username prefix before message
        client.send(message.encode("utf-8")) # Converts text into bytes.

threading.Thread(target=receive_messages).start() # Start background thread for receiving messages. Allows messages to appear while user types.
threading.Thread(target=write_messages).start() # Start another thread for sending messages.

## run this homemade whatsapp on different computers on the same WiFi campus net

You only need to change one key thing: instead of using 127.0.0.1 (localhost), you must use the server computer’s actual IP address on your campus WiFi network.

✅ 1. Pick ONE computer as the server

This computer runs:

python chat_server.py

It acts like the “WhatsApp server”.

⸻

✅ 2. Find the server computer’s IP address

On the server machine:

👉 Mac / Linux:

ifconfig

Look for something like:

inet 192.168.x.x

or

inet 10.x.x.x


⸻

👉 Windows:

ipconfig

Look for:

IPv4 Address

Example:

192.168.1.45


⸻

✅ 3. Modify the server code

Change:

HOST = "127.0.0.1"

to:

HOST = "0.0.0.0"

Why?

👉 This allows connections from other computers.

⸻

✅ 4. Modify ALL client files

Replace:

HOST = "127.0.0.1"

with:

HOST = "192.168.1.45"   # example — use your server IP


⸻

✅ 5. Allow firewall access (VERY important)

Sometimes campus networks block incoming connections.

Mac:

System Settings → Network → Firewall → allow Python.

Windows:

When prompted:

👉 Allow access on Private Networks.

⸻

✅ 6. Run the system

On server:

python chat_server.py


⸻

On each client computer:

python chat_client.py


⸻

🎉 Now:

All computers on the SAME WiFi should see each other.


